In [1]:
import pandas as pd
import numpy as np
import os
import xlrd

In [2]:
# delete duplicate file
def removeExist(path):
    if (os.path.exists(path)):
        os.remove(path)

In [3]:
########################################
# Trans currencies to USD base
def TransToUSDBase():
    currenciesPath = '../data/market/Currencies/'
    currencies = ['AUD_USD.csv', 'EUR_USD.csv', 'GBP_USD.csv']
    for currency in currencies:
        curPath = currenciesPath + currency
        newPath = currenciesPath + currency[4:7] + '_' + currency[:3] + '.csv'
        df = pd.read_csv(curPath)
        columns = [df.columns.values[1], df.columns.values[3], df.columns.values[4], df.columns.values[5]]
        for col in columns:
            df[col] = df[col].apply(lambda x: 1 / x)
        removeExist(newPath)
        df.to_csv(newPath, index=False, header=True)

In [4]:
# Sample: Initial structure of EUR_USD.csv
EUR_USD_path = '../data/market/Currencies/EUR_USD.csv'
EUR_USD = pd.read_csv(EUR_USD_path)
EUR_USD.head()

,Date,Close/Last,Volume,Open,High,Low
0,04/20/2020,1.0840,N/A,1.0862,1.0872,1.0825
1,04/19/2020,1.0859,N/A,1.0876,1.0878,1.0857
2,04/17/2020,1.0872,N/A,1.0835,1.0892,1.0813
3,04/16/2020,1.0869,N/A,1.0835,1.0880,1.0835
4,04/15/2020,1.0873,N/A,1.0907,1.0913,1.0873


In [5]:
# Execute DealWithCurrencies()
TransToUSDBase()

In [6]:
# Now we have USD to EUR currencies data
USD_EUR_path = '../data/market/Currencies/USD_EUR.csv'
USD_EUR = pd.read_csv(USD_EUR_path)
USD_EUR.head()

,Date,Close/Last,Volume,Open,High,Low
0,04/20/2020,0.922509,N/A,0.920641,0.919794,0.923788
1,04/19/2020,0.920895,N/A,0.919456,0.919287,0.921065
2,04/17/2020,0.919794,N/A,0.922935,0.918105,0.924813
3,04/16/2020,0.920048,N/A,0.922935,0.919118,0.922935
4,04/15/2020,0.919709,N/A,0.916842,0.916338,0.919709


In [7]:
########################################
# Trans Yahoo Data Format to NASDAQ Data Format
def TransYahooToNASDAQ():
    path_index = '../data/market/Index/'
    file_path = []
    file_path.append(path_index + 'CBOE_VolatilityIndex.csv')
    file_path.append(path_index + 'DowJones_IndustrialAvg.csv')
    file_path.append(path_index + 'NASDAQ_100_Index.csv')
    file_path.append(path_index + 'Russell_2000.csv')
    for path in file_path:
        df = pd.read_csv(path)
        # remove redundant column 'Adj Close'
        df = df.drop('Adj Close', axis=1, errors='ignore')
        # rename Close to Close/Last and put to second place
        if 'Close' in df.columns.values:
            df_close = df.Close
            df = df.drop('Close', axis=1, errors='ignore')
            df.insert(1, 'Close/Last', df_close)
        # reorder in time
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values('Date', inplace=True, ascending=False)
        # save
        df.to_csv(path, index=False, header=True)

In [8]:
# Sample: Initial structure of DowJones_IndustrialAvg in Yahoo format
DJI_yahoo_path = '../data/market/Index/DowJones_IndustrialAvg.csv'
DJI_yahoo = pd.read_csv(DJI_yahoo_path)
DJI_yahoo.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-04-21,18034.230469,18109.699219,17929.630859,17949.589844,17949.589844,95180000
1,2015-04-22,17950.820313,18056.019531,17887.759766,18038.269531,18038.269531,91260000
2,2015-04-23,18031.900391,18133.029297,17966.769531,18058.689453,18058.689453,100240000
3,2015-04-24,18056.419922,18108.869141,18009.080078,18080.140625,18080.140625,119130000
4,2015-04-27,18097.890625,18175.560547,18024.660156,18037.970703,18037.970703,121110000


In [9]:
# Execute TransYahooToNASDAQ()
TransYahooToNASDAQ()

In [10]:
# Now the structure of DowJones_IndustrialAvg is in NASDAQ format
DJI_yahoo = pd.read_csv(DJI_yahoo_path)
DJI_yahoo.head()

,Date,Close/Last,Open,High,Low,Volume
0,2020-04-20,23650.439453,24095.099609,24108.689453,23627.189453,423410000
1,2020-04-17,24242.490234,23817.150391,24264.210938,23817.150391,525950000
2,2020-04-16,23537.679688,23543.660156,23598.080078,23211.380859,471050000
3,2020-04-15,23504.349609,23600.720703,23649.720703,23233.320313,437630000
4,2020-04-14,23949.759766,23690.570313,24040.580078,23683.439453,485910000


In [11]:
########################################
# Clean Data From NASDAQ (Excepet All Stock Data)
def CleanDataFromNASDAQ():
    file_idx = 'NasdaqSource.txt'
    file_path = {}
    for line in open(file_idx):
        path = line.strip('\n')
        name = path.split("/")[-1][:-4]
        file_path[name] = path

    for name in file_path:
        print(name + ': ' + file_path[name])

    for path in file_path.values():
        df = pd.read_csv(path)
        # format column name, delete redundant whitespace in column names
        df.columns = [x.strip() for x in df.columns.values if x.strip() != '']
        # remove redundant column 'Volume'
        df = df.drop('Volume', axis=1, errors='ignore')
        # save
        df.to_csv(path, index=False, header=True)
        # print(path)
        # print(df.columns)

In [12]:
# Sample: Initial structure of NASDAQ_Composite
NASDAQ_comp_path = '../data/market/Index/NASDAQ_Composite.csv'
NASDAQ_comp = pd.read_csv(NASDAQ_comp_path)
NASDAQ_comp.head()

,Date,Close/Last,Volume,Open,High,Low
0,04/21/2020,8263.23,0,8460.69,8480.29,8215.69
1,04/20/2020,8560.73,0,8553.37,8684.91,8553.37
2,04/17/2020,8650.14,0,8667.48,8670.30,8531.69
3,04/16/2020,8532.36,0,8479.11,8560.16,8393.27
4,04/15/2020,8393.18,0,8355.96,8464.66,8308.79


In [13]:
# Execute CleanDataFromNASDAQ() and print all data collected from NASDAQ which will be modified
CleanDataFromNASDAQ()

BrentCrude: ../data/market/Commodities/Energies/BrentCrude.csv
CrudeOil_CL: ../data/market/Commodities/Energies/CrudeOil_CL.csv
Ethanol: ../data/market/Commodities/Energies/Ethanol.csv
Gasoline: ../data/market/Commodities/Energies/Gasoline.csv
HeatingOil: ../data/market/Commodities/Energies/HeatingOil.csv
NaturalGas: ../data/market/Commodities/Energies/NaturalGas.csv
Corn: ../data/market/Commodities/Grains/Corn.csv
Oat: ../data/market/Commodities/Grains/Oat.csv
RedWheat: ../data/market/Commodities/Grains/RedWheat.csv
RoughRice: ../data/market/Commodities/Grains/RoughRice.csv
SoybeanMeal: ../data/market/Commodities/Grains/SoybeanMeal.csv
SoybeanOil: ../data/market/Commodities/Grains/SoybeanOil.csv
Soybeans: ../data/market/Commodities/Grains/Soybeans.csv
Wheat: ../data/market/Commodities/Grains/Wheat.csv
FeederCattle: ../data/market/Commodities/Meats/FeederCattle.csv
LeanHogs: ../data/market/Commodities/Meats/LeanHogs.csv
LiveCattle: ../data/market/Commodities/Meats/LiveCattle.csv
Milk: 

In [14]:
# Now the redundant columns and blank space in column names are deleted
NASDAQ_comp = pd.read_csv(NASDAQ_comp_path)
NASDAQ_comp.head()

,Date,Close/Last,Open,High,Low
0,04/21/2020,8263.23,8460.69,8480.29,8215.69
1,04/20/2020,8560.73,8553.37,8684.91,8553.37
2,04/17/2020,8650.14,8667.48,8670.30,8531.69
3,04/16/2020,8532.36,8479.11,8560.16,8393.27
4,04/15/2020,8393.18,8355.96,8464.66,8308.79


In [15]:
########################################
# Clean Data From Other Source
def CleanOtherSources():
    file_CrudeOil_WTI_MacroTrends = '../data/market/Commodities/Energies/CrudeOil_WTI_macrotrends.csv'
    file_Finance_Sector_Related_Policy_Responses = '../data/general/FinanceSectorRelatedPolicyResponses.xlsx'
    file_Industrial_Production_Index = '../data/general/IndustrialProductionIndex.csv'
    file_Initial_Jobless_Claims = '../data/employment/InitialJoblessClaims.csv'
    file_Unemployment_Rate = '../data/employment/UnemploymentRate.csv'

    # Finance Sector Related Policy Responses
    FSRPR = pd.read_excel(file_Finance_Sector_Related_Policy_Responses, sheet_name='Raw data')
    # drop redundant columns
    FSRPR = FSRPR.drop('Iso 3 Code', axis=1, errors='ignore')
    FSRPR = FSRPR.drop('Entry date', axis=1, errors='ignore')
    # reorder
    FSRPR_Country = FSRPR.Country
    FSRPR = FSRPR.drop('Country', axis=1, errors='ignore')
    FSRPR.insert(0, 'Country', FSRPR_Country)
    FSRPR.to_csv(file_Finance_Sector_Related_Policy_Responses[:-4] + 'csv', index=False, header=True)

    # CrudeOil WTI MacroTrends
    CWM = pd.read_csv(file_CrudeOil_WTI_MacroTrends)
    # reorder in time
    CWM['date'] = pd.to_datetime(CWM['date'])
    CWM.sort_values('date', inplace=True, ascending=False)
    # format column name, delete redundant whitespace in column names
    CWM.columns = [x.strip() for x in CWM.columns.values if x.strip() != '']
    # delete invalid future data
    CWM.dropna(subset=['value'], inplace=True)
    CWM.to_csv(file_CrudeOil_WTI_MacroTrends, index=False, header=True)
    
    
    # Others with date (reorder date)
    file_path = []
    file_path.append(file_Industrial_Production_Index)
    file_path.append(file_Initial_Jobless_Claims)
    file_path.append(file_Unemployment_Rate)
    for path in file_path:
        df = pd.read_csv(path)
        # reorder in time
        df['DATE'] = pd.to_datetime(df['DATE'])
        df.sort_values('DATE', inplace=True, ascending=False)
        # save
        df.to_csv(path, index=False, header=True)

In [16]:
# Sample: Initial structure of Finance Sector Related Policy Responses
FSRPR_path = '../data/general/FinanceSectorRelatedPolicyResponses.xlsx'
FSRPR = pd.read_excel(FSRPR_path, sheet_name='Raw data')
FSRPR.head()

,Iso 3 Code,Level of income,Region,Country,Date (at or prior to),Entry date,Level 1 policy measure,Level 2 policy measure
0,CHN,Upper middle income,EAP,China,2020-02-01,2020-03-21,Financial Institutions,Operational continuity
1,CAN,High income,Other G20,Canada,2020-02-03,2020-03-21,Financial Markets,Market functioning
2,THA,Upper middle income,EAP,Thailand,2020-02-05,2020-03-21,Liquidity/funding,Policy rate
3,RUS,Upper middle income,ECA,Russian Federation,2020-02-07,2020-03-21,Liquidity/funding,Policy rate
4,BLR,Upper middle income,ECA,Belarus,2020-02-19,2020-03-31,Liquidity/funding,Liquidity (incl FX)/ELA


In [17]:
# Execute CleanOtherSources()
CleanOtherSources()

In [18]:
# Now the file is in csv format and redundant columns are deleted
FSRPR = pd.read_csv('../data/general/FinanceSectorRelatedPolicyResponses.csv')
FSRPR.head()

,Country,Level of income,Region,Date (at or prior to),Level 1 policy measure,Level 2 policy measure
0,China,Upper middle income,EAP,2020-02-01,Financial Institutions,Operational continuity
1,Canada,High income,Other G20,2020-02-03,Financial Markets,Market functioning
2,Thailand,Upper middle income,EAP,2020-02-05,Liquidity/funding,Policy rate
3,Russian Federation,Upper middle income,ECA,2020-02-07,Liquidity/funding,Policy rate
4,Belarus,Upper middle income,ECA,2020-02-19,Liquidity/funding,Liquidity (incl FX)/ELA
